In [82]:
df = spark.sql("SELECT * FROM binglakedb.tbl_latest_fraudnews")
display(df)

StatementMeta(, de70dad5-3f02-4f3b-874e-03d6ed766d8d, 84, Finished, Available)

SynapseWidget(Synapse.DataFrame, c3195f7e-16f1-4d35-aaf8-692f3d8911e4)

In [83]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, de70dad5-3f02-4f3b-874e-03d6ed766d8d, 85, Finished, Available)

In [84]:
model=(AnalyzeText()
    .setTextCol("description")
    .setKind("SentimentAnalysis")
    .setOutputCol("response")
    .setErrorCol("error")
    )

StatementMeta(, de70dad5-3f02-4f3b-874e-03d6ed766d8d, 86, Finished, Available)

In [85]:
# Display the source data
df = spark.sql("SELECT * FROM binglakedb.tbl_latest_fraudnews")
display(df.select("url", "datePublished"))

StatementMeta(, de70dad5-3f02-4f3b-874e-03d6ed766d8d, 87, Finished, Available)

SynapseWidget(Synapse.DataFrame, 4e11f56a-81a2-44fb-9972-5090afb3dde2)

In [86]:
result = model.transform(df)          #coverts  into json format

StatementMeta(, de70dad5-3f02-4f3b-874e-03d6ed766d8d, 88, Finished, Available)

In [87]:
display(result)

StatementMeta(, de70dad5-3f02-4f3b-874e-03d6ed766d8d, 89, Finished, Available)

SynapseWidget(Synapse.DataFrame, c2791c73-2557-4927-b363-252fe0f55e86)

In [88]:
from pyspark.sql.functions import col
fraud_df_final=result.withColumn("opinionmining", col("response.documents.sentiment"))

StatementMeta(, de70dad5-3f02-4f3b-874e-03d6ed766d8d, 90, Finished, Available)

In [89]:
display(fraud_df_final)

StatementMeta(, de70dad5-3f02-4f3b-874e-03d6ed766d8d, 91, Finished, Available)

SynapseWidget(Synapse.DataFrame, b92312bd-38f4-426c-9b85-e871108ebed4)

In [90]:
fraud_df_final = fraud_df_final.drop("error", "response")

StatementMeta(, de70dad5-3f02-4f3b-874e-03d6ed766d8d, 92, Finished, Available)

In [91]:
display(fraud_df_final)               #Final data

StatementMeta(, de70dad5-3f02-4f3b-874e-03d6ed766d8d, 93, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9f7b1f5f-693a-46e8-88ce-be3706914e0a)

In [92]:
# Save Processed Data to Delta Table
from pyspark.sql.utils import AnalysisException

# Table name
table_name = 'binglakedb.tbl_sentiment_analysis'

# Try to save the DataFrame as a Delta table
try:
    fraud_df_final.write.format("delta").saveAsTable(table_name)
except AnalysisException as e:
    print("Table Already Exists:", e)

fraud_df_final.createOrReplaceTempView("vw_fraud_df_final")

# Define the columns explicitly for the source view
merge_query = f"""
MERGE INTO {table_name} target_table
USING (
    SELECT url,
           MAX(title) AS title,
           MAX(description) AS description,
           MAX(category) AS category,
           MAX(image) AS image,
           MAX(provider) AS provider,
           MAX(datePublished) AS datePublished,
           MAX(opinionmining) AS opinionmining
    FROM vw_fraud_df_final
    GROUP BY url
) AS source_view
ON source_view.url = target_table.url
WHEN MATCHED AND (
    source_view.title <> target_table.title OR
    source_view.description <> target_table.description OR
    source_view.category <> target_table.category OR
    source_view.image <> target_table.image OR
    source_view.provider <> target_table.provider OR
    source_view.datePublished <> target_table.datePublished OR
    source_view.opinionmining <> target_table.opinionmining
)
THEN UPDATE SET
    title = source_view.title,
    description = source_view.description,
    category = source_view.category,
    image = source_view.image,
    provider = source_view.provider,
    datePublished = source_view.datePublished,
    opinionmining = source_view.opinionmining
WHEN NOT MATCHED THEN INSERT (url, title, description, category, image, provider, datePublished, opinionmining)
VALUES (source_view.url, source_view.title, source_view.description, source_view.category, source_view.image, source_view.provider, source_view.datePublished, source_view.opinionmining)
"""

spark.sql(merge_query)


StatementMeta(, de70dad5-3f02-4f3b-874e-03d6ed766d8d, 94, Finished, Available)

Table Already Exists: [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `binglakedb`.`tbl_sentiment_analysis` because it already exists.
Choose a different name, drop or replace the existing object, or add the IF NOT EXISTS clause to tolerate pre-existing objects.


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [93]:
 %%sql
 select * from tbl_sentiment_analysis

StatementMeta(, de70dad5-3f02-4f3b-874e-03d6ed766d8d, 95, Finished, Available)

<Spark SQL result set with 533 rows and 8 fields>